In [1]:
import scanpy as sc
import pandas as pd
from matplotlib import pylab
import random
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import itertools
import numpy as np
import random
import PyWGCNA
import anndata as ad
from scipy import sparse

from matplotlib.colors import TwoSlopeNorm

import scanpy.external as sce

/group/testa/Users/davide.castaldi/Polaroid_publication_revisions/.local/lib/python3.8/site-packages/setuptools_scm/_integration/setuptools.py:30: RuntimeWarning: 
ERROR: setuptools==56.1.0 is used in combination with setuptools_scm>=8.x

Your build configuration is incomplete and previously worked by accident!
setuptools_scm requires setuptools>=61

Suggested workaround if applicable:
 - migrating from the deprecated setup_requires mechanism to pep517/518
   and using a pyproject.toml to declare build dependencies
   which are reliably pre-installed before running the build tools

  warnings.warn(


In [2]:
sc.settings.set_figure_params(dpi=100, facecolor='white', dpi_save=500)
pylab.rcParams['figure.figsize'] = (9, 9)


In [4]:
xgenes = pd.read_csv("./data/resources/X.genes.tsv", header=None)[0].tolist()
ygenes = pd.read_csv("./data/resources/Y.genes.tsv", header=None)[0].tolist()

# PolCAs 

In [6]:
polCAs = sc.read("./outdir/3_polaroid_quickAnno.h5ad")
polCAsRAws = sc.read_h5ad("./outdir/1_polaroid_mint.h5ad")[polCAs.obs_names]
sc.pp.normalize_total(polCAsRAws,target_sum=20000, exclude_highly_expressed=True)
polCAsRAws.obs["Protocol"] = "polCAs"
polCAsRAws.obs["sample"] = polCAsRAws.obs["organoid"]
polCAsRAws.obs["OriginAnnotation"] = polCAs.obs.loc[polCAsRAws.obs_names,"leidenAnno"]


# Velasco

In [8]:
Velasco_3M_B1 = sc.read_text("./data/Velasco/Velasco_2019_3Month_B1.txt.gz").T
Velasco_3M_B1.X = sparse.csr_matrix(Velasco_3M_B1.X)
Velasco_3M_B2 = sc.read_text("./data/Velasco/Velasco_2019_3Month_B2.txt.gz").T
Velasco_3M_B2.X = sparse.csr_matrix(Velasco_3M_B2.X)
Velasco_3M_B2.obs["Replicate"] = "B2"
Velasco_3M_B1.obs["Replicate"] = "B1"
VelascoCombined = ad.concat([Velasco_3M_B2, Velasco_3M_B1])
MD = pd.read_csv("./data/Velasco/meta_combined.txt", sep="\t", index_col=0)
MD = MD[MD.Batch != "group"]
MD.index.name = None

VelascoCombined.obs = pd.concat([VelascoCombined.obs,MD], axis = 1).dropna().loc[VelascoCombined.obs.index.tolist()]
VelascoCombined.obs["sample"] = VelascoCombined.obs["Replicate"]
VelascoCombined.obs["Protocol"] = "Velasco"
VelascoCombined.obs["Protocol_sample"] = VelascoCombined.obs["Protocol"].astype("str") + "_" + VelascoCombined.obs["sample"].astype("str")
VelascoCombined.obs["OriginAnnotation"] = VelascoCombined.obs["CellType"]
# Revert to 20k normalized counts (authors normalized with target 1M counts)
VelascoCombined.X = np.expm1(VelascoCombined.X) / 50
VelascoCombined = VelascoCombined[VelascoCombined.obs["OriginAnnotation"].isin(["CFuPNs","Cycling","RG","CPNs"])]


In [9]:

VelascoCombined.obs.OriginAnnotation.unique().tolist()

['CFuPNs', 'Cycling', 'RG', 'CPNs']

In [10]:
# Check common genes
len(VelascoCombined.var_names.intersection(polCAsRAws.var_names))


22161

# Kanton

In [12]:


Kanton = sc.read_10x_mtx("./data/Kanton/")
obs = ["CB","Species","Stage","Line","Sample","Barcode","PredCellType","nGene","nUMI","PercentMito","in_FullLineage","in_LineComp","in_DE","pseudocellIdx_FullLineage","pseudocellIdx_LineComp","pseudocellIdx_DE","cl_FullLineage","cl_LineComp"]
obs = ["CB","Species","Stage","Line","Sample","Barcode","PredCellType","nGene","nUMI","PercentMito","in_FullLineage","in_LineComp","in_DE","pseudocellIdx_FullLineage","pseudocellIdx_LineComp","pseudocellIdx_DE","cl_FullLineage","cl_LineComp"]
obs = pd.read_csv("./data/Kanton/barcodes.tsv", delimiter="\t", header=None, names=obs, index_col=0)
obs.index.name = None
obs
Kanton.obs = obs
Kanton.obs["Protocol"] = "Kanton"
Kanton = Kanton[Kanton.obs.Stage.str.contains("Organoid-2M")]
Kanton = Kanton[~Kanton.obs["Sample"].str.contains("HipSci")]
sc.pp.normalize_total(Kanton,target_sum=20000, exclude_highly_expressed=True)



Kanton.obs["sample"] = Kanton.obs["Sample"]
Kanton.obs["Protocol"] = "Kanton"

Kanton = Kanton[Kanton.obs.cl_FullLineage.isin(["cycling dorsal progenitors","cortical neurons 1","RGCs early 2","RGCs 3","IPs and early cortical neurons"])]
Kanton.obs["OriginAnnotation"] = Kanton.obs["cl_FullLineage"]

# Concat

In [13]:
OrganoidsAll = ad.concat([VelascoCombined,polCAsRAws,Kanton ])

In [14]:
OrganoidsAll = OrganoidsAll[:,~OrganoidsAll.var_names.isin(xgenes+ygenes)]

In [15]:
OrganoidsAll.write_h5ad("./data/AllOrganoids.h5ad")

In [16]:
OrganoidsAll


AnnData object with n_obs × n_vars = 50529 × 21219
    obs: 'sample', 'Protocol', 'OriginAnnotation'